In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# --- Definisi state dan action ---
states = [
"Panas-Ada", "Nyaman-Ada", "Dingin-Ada",
"Panas-TidakAda", "Nyaman-TidakAda", "Dingin-TidakAda"
]
actions = ["Matikan", "Diam", "Nyalakan"]


state_size = len(states)
action_size = len(actions)


# --- Inisialisasi Q-Table ---
q_table = np.zeros((state_size, action_size))

In [2]:
# --- Hiperparameter ---
alpha = 0.9 # learning rate
gamma = 0.8 # discount factor
epsilon = 1.0 # eksplorasi awal
min_epsilon = 0.01
decay_rate = 0.005

In [8]:
# --- Lingkungan (sederhana) ---
def lingkungan_sederhana(state, state_index, action_index):
    suhu, hadir = state.split("-")
    action = actions[action_index]

    # default
    reward = -1
    next_state = state_index

    # logika transisi
    if suhu == "Panas" and hadir == "Ada":
        if action == "Nyalakan":
            next_state = states.index("Nyaman-Ada")
            reward = 10
        elif action == "Diam":
            next_state = state_index
            reward = -5
        else:  # Matikan
            next_state = states.index("Panas-Ada")
            reward = -10

    elif suhu == "Dingin" and hadir == "Ada":
        if action == "Matikan":
            next_state = states.index("Nyaman-Ada")
            reward = 10
        elif action == "Diam":
            next_state = state_index
            reward = -5
        else:  # Nyalakan
            next_state = states.index("Dingin-Ada")
            reward = -10

    elif suhu == "Nyaman" and hadir == "Ada":
        if action == "Diam":
            next_state = state_index
            reward = 5
        elif action == "Nyalakan":
            next_state = states.index("Dingin-Ada")
            reward = -10
        else:
            next_state = states.index("Panas-Ada")
            reward = -10

    # kondisi tidak ada orang -> penalti untuk pemborosan
    if hadir == "TidakAda":
        if action == "Nyalakan":
            reward = -5
        elif action == "Diam":
            reward = 2
        else:  # Matikan
            reward = 5

        # transisi sederhana: tanpa orang, suhu cenderung stabil
        if suhu == "Panas":
            next_state = states.index("Nyaman-TidakAda")
        elif suhu == "Dingin":
            next_state = states.index("Nyaman-TidakAda")
        else:
            next_state = state_index

    return next_state, reward

In [ ]:
# --- Pelatihan ---
total_episodes = 10000
max_steps = 10
rewards_all_episodes = []

for episode in range(total_episodes):
    state = np.random.randint(0, state_size)
    rewards_current_episode = 0

    for step_num in range(max_steps):
        if np.random.uniform(0, 1) < epsilon:
            action = np.random.randint(0, action_size)
        else:
            action = np.argmax(q_table[state, :])

        new_state, reward = lingkungan_sederhana(states[state], state, action)

        # update Q
        old_value = q_table[state, action]
        next_max = np.max(q_table[new_state, :])
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        state = new_state
        rewards_current_episode += reward

    # decay epsilon
    epsilon = min_epsilon + (1.0 - min_epsilon) * np.exp(-decay_rate * episode)
    rewards_all_episodes.append(rewards_current_episode)

NameError: name 'step' is not defined